In [1]:
import sys
import os
from dotenv import load_dotenv


load_dotenv()

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

In [2]:
from langchain_openai import ChatOpenAI
from helper_functions import get_local_embedding_model


llm = ChatOpenAI(model="qwen-max", temperature=0)
embedding_model = get_local_embedding_model("C:\\Home\\Documents\\Projects\\models\\BAAI\\bge-large-en-v1.5")

# 文档加载与切分

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader


urls = [
    "https://www.deeplearning.ai/the-batch/how-agents-can-improve-llm-performance/?ref=dl-staging-website.ghost.io",
    "https://www.deeplearning.ai/the-batch/agentic-design-patterns-part-2-reflection/?ref=dl-staging-website.ghost.io",
    "https://www.deeplearning.ai/the-batch/agentic-design-patterns-part-3-tool-use/?ref=dl-staging-website.ghost.io",
    "https://www.deeplearning.ai/the-batch/agentic-design-patterns-part-4-planning/?ref=dl-staging-website.ghost.io",
    "https://www.deeplearning.ai/the-batch/agentic-design-patterns-part-5-multi-agent-collaboration/?ref=dl-staging-website.ghost.io",
]

# Load
docs = [WebBaseLoader(url).load() for url in urls]
docs_list = [item for sublist in docs for item in sublist]

# Split
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=500, chunk_overlap=0
)

doc_splits = text_splitter.split_documents(docs_list)

len(doc_splits), len(doc_splits[0].page_content)

USER_AGENT environment variable not set, consider setting it to identify your requests.


(13, 2228)

# 创建向量存储

In [4]:
from uuid import uuid4
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams


client = QdrantClient(host="127.0.0.1", port=6333)

collection_name = "reliable_rag"

if not client.collection_exists(collection_name=collection_name):
    client.create_collection(
        collection_name=collection_name,
        vectors_config=VectorParams(size=1024, distance=Distance.COSINE),
    )

vector_store = QdrantVectorStore(
    client=client,
    collection_name=collection_name,
    embedding=embedding_model,
)

ids = [str(uuid4()) for _ in range(len(doc_splits))]
vector_store.add_documents(documents=doc_splits, ids=ids)

['6399c3ae-4159-4ac5-809a-55f25959a40d',
 'a769cd34-0075-4b37-9ef7-d7d604db29ac',
 'd15298a5-a260-49cd-b383-e25b275d479d',
 'c1b70e72-c593-48de-988a-c666ff14a719',
 '61aa95df-2860-419d-bd66-29f8490c9626',
 '2c1985b8-cade-4623-ab6f-9b8c16bc07ea',
 'aabce515-81af-4767-895c-51353973db92',
 '028fe356-3950-424e-9a65-d2b6f3c7b3ed',
 'dc407a25-668f-4751-bd27-ac4447855698',
 '4d784ba1-ad9c-48fb-8afb-c1d1a73e2698',
 '1da1f15a-50f1-4bba-a35c-f5df17ed6470',
 '915f928b-3ed5-4de9-b445-6cdf1bb5be77',
 '74b7849d-3dda-4592-a10a-c5d64b553d33']

# 检索

In [5]:
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 4})

In [6]:
question = "what are the differnt kind of agentic design patterns?"

docs = retriever.invoke(question)

print(
    f"Title: {docs[0].metadata['title']}\n\nSource: {docs[0].metadata['source']}\n\nContent: {docs[0].page_content}\n"
)

Title: Agentic Design Patterns Part 4: Planning

Source: https://www.deeplearning.ai/the-batch/agentic-design-patterns-part-4-planning/?ref=dl-staging-website.ghost.io

Content: agents: A survey,” by Huang et al. (2024)Keep learning!AndrewRead "Agentic Design Patterns Part 1: Four AI agent strategies that improve GPT-4 and GPT-3.5 performance"Read "Agentic Design Patterns Part 2: Reflection" Read "Agentic Design Patterns Part 3: Tool Use"Read "Agentic Design Patterns Part 5: Multi-Agent Collaboration"ShareSubscribe to The BatchStay updated with weekly AI News and Insights delivered to your inboxCoursesThe BatchCommunityCareersAbout



# 文档相关性检查

In [7]:
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field


# Data model
class GradeDocuments(BaseModel):
    """Binary score for relevance check on retrieved documents."""

    binary_score: str = Field(
        description="Documents are relevant to the question, 'yes' or 'no'"
    )


# LLM with function call
llm_grader = ChatOpenAI(model="qwen-plus", temperature=0)
structured_llm_grader = llm_grader.with_structured_output(GradeDocuments)

# Prompt
system = """You are a grader assessing relevance of a retrieved document to a user question. \n 
    If the document contains keyword(s) or semantic meaning related to the user question, grade it as relevant. \n
    It does not need to be a stringent test. The goal is to filter out erroneous retrievals. \n
    Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question."""

grade_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "Retrieved document: \n\n {document} \n\n User question: {question}"),
    ]
)

retrieval_grader = grade_prompt | structured_llm_grader

In [8]:
docs_to_use = []

for doc in docs:
    print(doc.page_content, "\n", "-" * 50)
    res = retrieval_grader.invoke(
        {"question": question, "document": doc.page_content}
    )
    print(res, "\n")
    if res.binary_score == "yes":
        docs_to_use.append(doc)

agents: A survey,” by Huang et al. (2024)Keep learning!AndrewRead "Agentic Design Patterns Part 1: Four AI agent strategies that improve GPT-4 and GPT-3.5 performance"Read "Agentic Design Patterns Part 2: Reflection" Read "Agentic Design Patterns Part 3: Tool Use"Read "Agentic Design Patterns Part 5: Multi-Agent Collaboration"ShareSubscribe to The BatchStay updated with weekly AI News and Insights delivered to your inboxCoursesThe BatchCommunityCareersAbout 
 --------------------------------------------------
binary_score='yes' 

Multimodal Reasoning and Action,” Yang et al. (2023)“Efficient Tool Use with Chain-of-Abstraction Reasoning,” Gao et al. (2024)   Both Tool Use and Reflection, which I described in last week’s letter, are design patterns that I can get to work fairly reliably on my applications — both are capabilities well worth learning about. In future letters, I’ll describe the Planning and Multi-agent collaboration design patterns. They allow AI agents to do much more but 

# 生成答案

In [9]:
from langchain_core.output_parsers import StrOutputParser

# Prompt
system = """You are an assistant for question-answering tasks. Answer the question based upon your knowledge. 
Use three-to-five sentences maximum and keep the answer concise."""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        (
            "human",
            "Retrieved documents: \n\n <docs>{documents}</docs> \n\n User question: <question>{question}</question>",
        ),
    ]
)


# Post-processing
def format_docs(docs):
    return "\n".join(
        f"<doc{i + 1}>:\nTitle:{doc.metadata['title']}\nSource:{doc.metadata['source']}\nContent:{doc.page_content}\n</doc{i + 1}>\n"
        for i, doc in enumerate(docs)
    )


# Chain
rag_chain = prompt | llm | StrOutputParser()

In [10]:
# Run
generation = rag_chain.invoke(
    {"documents": format_docs(docs_to_use), "question": question}
)

print(generation)

The different kinds of agentic design patterns include Reflection, Tool Use, Planning, and Multi-Agent Collaboration. These patterns are strategies to enhance the performance and capabilities of AI agents. Each pattern focuses on a specific aspect, such as improving self-awareness, utilizing tools effectively, planning tasks, and coordinating multiple agents for complex tasks.


# 幻觉检测

In [11]:
# Data model
class GradeHallucinations(BaseModel):
    """Binary score for hallucination present in 'generation' answer."""

    binary_score: str = Field(
        ..., description="Answer is grounded in the facts, 'yes' or 'no'"
    )


# LLM with function call
structured_llm_grader = llm_grader.with_structured_output(GradeHallucinations)

# Prompt
system = """You are a grader assessing whether an LLM generation is grounded in / supported by a set of retrieved facts. \n 
    Give a binary score 'yes' or 'no'. 'Yes' means that the answer is grounded in / supported by the set of facts."""

hallucination_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        (
            "human",
            "Set of facts: \n\n <facts>{documents}</facts> \n\n LLM generation: <generation>{generation}</generation>",
        ),
    ]
)

hallucination_grader = hallucination_prompt | structured_llm_grader

In [12]:
response = hallucination_grader.invoke(
    {"documents": format_docs(docs_to_use), "generation": generation}
)

print(response)

binary_score='yes'


# 文档片段高亮

In [13]:
from typing import List
from langchain.output_parsers import PydanticOutputParser


# Data model
class HighlightDocuments(BaseModel):
    """Return the specific part of a document used for answering the question."""

    id: List[str] = Field(
        ..., description="List of id of docs used to answers the question"
    )

    title: List[str] = Field(
        ..., description="List of titles used to answers the question"
    )

    source: List[str] = Field(
        ..., description="List of sources used to answers the question"
    )

    segment: List[str] = Field(
        ...,
        description="List of direct segements from used documents that answers the question",
    )


parser = PydanticOutputParser(pydantic_object=HighlightDocuments)

In [14]:
from langchain_core.prompts import PromptTemplate


system = """You are an advanced assistant for document search and retrieval. You are provided with the following:
1. A question.
2. A generated answer based on the question.
3. A set of documents that were referenced in generating the answer.

Your task is to identify and extract the exact inline segments from the provided documents that directly correspond to the content used to 
generate the given answer. The extracted segments must be verbatim snippets from the documents, ensuring a word-for-word match with the text 
in the provided documents.

Ensure that:
- (Important) Each segment is an exact match to a part of the document and is fully contained within the document text.
- The relevance of each segment to the generated answer is clear and directly supports the answer provided.
- (Important) If you didn't used the specific document don't mention it.

Used documents: <docs>{documents}</docs> \n\n User question: <question>{question}</question> \n\n Generated answer: <answer>{generation}</answer>

<format_instruction>
{format_instructions}
</format_instruction>
"""


prompt = PromptTemplate(
    template=system,
    input_variables=["documents", "question", "generation"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

# Chain
doc_lookup = prompt | llm | parser

In [15]:
# Run
lookup_response = doc_lookup.invoke(
    {
        "documents": format_docs(docs_to_use),
        "question": question,
        "generation": generation,
    }
)

In [16]:
for id, title, source, segment in zip(
    lookup_response.id,
    lookup_response.title,
    lookup_response.source,
    lookup_response.segment,
):
    print(f"ID: {id}\nTitle: {title}\nSource: {source}\nText Segment: {segment}\n")

ID: doc2
Title: Agentic Design Patterns Part 3: Tool Use
Source: https://www.deeplearning.ai/the-batch/agentic-design-patterns-part-3-tool-use/?ref=dl-staging-website.ghost.io
Text Segment: Both Tool Use and Reflection, which I described in last week’s letter, are design patterns that I can get to work fairly reliably on my applications — both are capabilities well worth learning about. In future letters, I’ll describe the Planning and Multi-agent collaboration design patterns.

ID: doc3
Title: Agentic Design Patterns Part 5, Multi-Agent Collaboration
Source: https://www.deeplearning.ai/the-batch/agentic-design-patterns-part-5-multi-agent-collaboration/?ref=dl-staging-website.ghost.io
Text Segment: Multi-agent collaboration is the last of the four key AI agentic design patterns that I’ve described in recent letters.

